In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
state_names = {
    "U01": "Andaman & Nicobar Islands", "U02": "Chandigarh", "U03": "Dadra & Nagar Haveli and Daman & Diu", "U04": "State 4", 
    "U05": "NCT OF Delhi", "U06": "Lakshadweep", "U07": "Puducherry", "S01": "Andhra Pradesh", "S02": "Arunachal Pradesh", 
    "S03": "Assam", "S04": "Bihar", "S05": "Goa", "S06": "Gujarat", "S07": "Haryana", "S08": "Himachal Pradesh", 
    "S09": "State 16", "S10": "Karnataka", "S11": "Kerala", "S12": "Madhya Pradesh", "S13": "Maharashtra", 
    "S14": "Manipur", "S15": "Meghalaya", "S16": "Mizoram", "S17": "Nagaland", "S18": "Odisha", 
    "S19": "Punjab", "S20": "Rajasthan", "S21": "Sikkim", "S22": "Tamil Nadu", "S23": "Tripura", 
    "S24": "Uttar Pradesh", "S25": "West Bengal", "S26": "Chhattisgarh", "S27": "Jharkhand", "S28": "Uttarakhand", "S29": "Telangana"
}
states = list(state_names.keys())
base_url = "http://results.eci.gov.in/PcResultGenJune2024/Constituencywise"

In [17]:
results = []

for state in states:
    successful_fetches = 0
    print(f"Processing state: {state_names[state]}")
    
    for constituency_code in range(1, 99):
        url = base_url + state + str(constituency_code) + ".htm"
        response = requests.get(url)

        if response.status_code == 404:
            break

        response_text = response.text
        soup = BeautifulSoup(response_text, 'html.parser')
        
        constituency_name_tag = soup.find('h2')
        if constituency_name_tag:
            constituency_name = constituency_name_tag.find('span').text.split('-')[-1].strip()
        else:
            constituency_name = "Unknown"
        
        table = soup.find('table', class_='table table-striped table-bordered')
        
        if table:
            for row in table.find('tbody').find_all('tr'):
                cols = row.find_all('td')
                
                if len(cols) == 7:  
                    candidate = cols[1].text.strip()
                    party = cols[2].text.strip()
                    evm_votes = cols[3].text.strip()
                    postal_votes = cols[4].text.strip()
                    total_votes = cols[5].text.strip()
                    percent_votes = cols[6].text.strip()

                    results.append({
                        'State': state_names[state],
                        'Constituency No.': constituency_code,
                        'Constituency Name': constituency_name,
                        'Candidate': candidate,
                        'Party': party,
                        'EVM Votes': evm_votes,
                        'Postal Votes': postal_votes,
                        'Total Votes': total_votes,
                        '% of Votes': percent_votes
                    })
            successful_fetches += 1
    
    print(f"Finished processing state: {state_names[state]} with {successful_fetches} successful constituency fetches.")
    df = pd.DataFrame(results)
    df['Total Votes'] = pd.to_numeric(df['Total Votes'].str.replace(',', ''), errors='coerce')


    df['Winner'] = df.groupby(['State', 'Constituency No.'])['Total Votes'].transform('max') == df['Total Votes']

Processing state: Andaman & Nicobar Islands
Finished processing state: Andaman & Nicobar Islands with 1 successful constituency fetches.
Processing state: Chandigarh
Finished processing state: Chandigarh with 1 successful constituency fetches.
Processing state: Dadra & Nagar Haveli and Daman & Diu
Finished processing state: Dadra & Nagar Haveli and Daman & Diu with 2 successful constituency fetches.
Processing state: State 4
Finished processing state: State 4 with 0 successful constituency fetches.
Processing state: NCT OF Delhi
Finished processing state: NCT OF Delhi with 7 successful constituency fetches.
Processing state: Lakshadweep
Finished processing state: Lakshadweep with 1 successful constituency fetches.
Processing state: Puducherry
Finished processing state: Puducherry with 1 successful constituency fetches.
Processing state: Andhra Pradesh
Finished processing state: Andhra Pradesh with 25 successful constituency fetches.
Processing state: Arunachal Pradesh
Finished processi

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [8]:
df.to_csv('election_results.csv', index=False)